In [ ]:
# Install and import necessary libraries
%pip install -qU sentencepiece accelerate bitsandbytes pydantic hypertion

import json
import torch
from hypertion import HyperFunction
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
)

### Create a `HyperFunction` instance

In [4]:
hyperfunction = HyperFunction()

### Define function and register it using `.takeover` method

In [1]:
import pathlib
from enum import Enum
from typing import Literal, NamedTuple
from typing_extensions import TypedDict
from pydantic import BaseModel, Field


class ServerType(str, Enum):
    TESTING = "testing"
    DEVELOPMENT = "development"
    PRODUCTION = "production"

class ServerConfig(BaseModel):
    host: str = Field(default="http://localhost", description="The host url of the server.")
    port: int = Field(..., description="The port of the server.")
    use_ssl: bool = False

class LoggingConfig(TypedDict):
    """
    Logging Configuration
    :param level: Level of the logging.
    :param format: Logging format.
    """
    level: Literal['debug', 'info', 'warning', 'error', 'critical'] = 'info'
    format: str

class User(NamedTuple):
    """
    User Info
    :param name: Name of the user.
    :param role: Role of the user.
    """
    name: str
    role: Literal['admin', 'developer', 'tester'] = 'developer'

@hyperfunction.takeover
def setup_server(
    name: str,
    server_type: ServerType,
    server_config: ServerConfig,
    log_config: LoggingConfig,
    users: list[User],
    persist_at: pathlib.Path,
) -> None:
    """
    Setup server with the given configuration.

    @param name: The name of the server.
    @param server_type: The type of the server.
    @param server_config: Configuration for the server.
    @param log_config: Logging configuration.
    @param users: List of users with their respective roles.
    @param persist_at: Path to persist server log.
    """
    print(f"Setting up server '{name}' of type '{server_type}'...")
    print(f"Server Configuration: {server_config}")
    print(f"Logging Configuration: {log_config}")
    print(f"Users: {users}")
    print(f"Persist location: {persist_at}")

### Display registered functions

In [2]:
hyperfunction.registry()

setup_server(
   name: str,
   server_type: ServerType,
   server_config: ServerConfig,
   log_config: LoggingConfig,
   users: list[User],
   persist_at: Path
) -> None:
"""Setup server with the given configuration."""


### Build function schema with specified LLM format

In [3]:
print(hyperfunction.format('gorilla', as_json=True))

[
    {
        "api_call": "setup_server",
        "name": "setup_server",
        "description": "Setup server with the given configuration.",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "The name of the server."
                },
                "server_type": {
                    "type": "string",
                    "enum": [
                        "TESTING",
                        "DEVELOPMENT",
                        "PRODUCTION"
                    ],
                    "description": "The type of the server."
                },
                "server_config": {
                    "type": "object",
                    "properties": {
                        "host": {
                            "type": "string",
                            "description": "The host url of the server."
                        },
                        "

### Setup pipeline

In [ ]:
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# Quantization Setup
# This configuration compresses the model and makes it possible to use with Colab's GPU
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype
)

# Model and tokenizer setup
model_id : str = "gorilla-llm/gorilla-openfunctions-v2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    quantization_config=bnb_config,
    low_cpu_mem_usage=True,
    device_map="auto"
)

# Create pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    batch_size=16,
    torch_dtype=torch_dtype,
)

# This function will return Gorilla's response
def get_gorilla_signature(query: str, functions: list[dict]) -> str:
    output = pipe(
        f"USER: <<question>> {query} <<function>> {json.dumps(functions)}\nASSISTANT: "
    )[0]['generated_text'].splitlines()[-1]
    return output[output.index(":")+1:].strip().strip('<<function>>')

### Invoke `Gorilla` and get the function signature

In [ ]:
QUERY = """\
Setup a server of type development on port number 8001 with ssl enabled.
Logging format should be "%(asctime)s - %(name)s - %(levelname)s - %(message)s".
Add user synacktra with developer role and 1littlecoder with tester role.
The server logs should be persisted at /var/logs.\
"""
signature = get_gorilla_signature(query=QUERY, functions=hyperfunction.format('gorilla'))
print(signature)

### Compose the signature as Function-Call object

In [ ]:
function_call = hyperfunction.compose(signature)
print(function_call)

### Invoke the function

In [ ]:
function_call()